# TUAB

This notebook looks around the Temple University Hospital EEG Abnormal Corpus (`TUAB`) dataset and convert the file format from the `EDF` to `NumPy memmap` for the speed-up.

-----

## Configure environments

In [ ]:
# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2
%cd ..

In [ ]:
# Load some packages
import os
import glob

import math
import json
import pyedflib
import numpy as np

import pprint
from tqdm.auto import tqdm

# custom package
from datasets.temple_eeg_dataset import *
from datasets.temple_eeg_script import build_dataset_for_tuab_train
from datasets.pipeline import *

---
## `TUAB` dataset

In [ ]:
# Data file path
origin_path = r'H:\Other_DB\Temple_EEG\tuh_eeg_abnormal\v2.0.0\edf'
desired_path = r'local/dataset/tuab/'

### Age

In [ ]:
count = 0
text_files = glob.glob(os.path.join(origin_path, 'train/normal/01_tcp_ar/*/*/*/*.txt'))
for filename in text_files:
    with open(filename, 'rt', encoding='UTF-8') as f:
        text_script = f.read()
        if 'year' in text_script:
            count += 1
        else:
            print(text_script)
            print('-----' * 4)
            
print(count, len(text_files)) 

## File Conversion to MEMMAP

In [ ]:
ref_headers = ['EEG FP1-REF', 'EEG FP2-REF', 'EEG F3-REF', 'EEG F4-REF', 
               'EEG C3-REF', 'EEG C4-REF', 'EEG P3-REF', 'EEG P4-REF', 
               'EEG O1-REF', 'EEG O2-REF', 'EEG F7-REF', 'EEG F8-REF', 
               'EEG T3-REF', 'EEG T4-REF', 'EEG T5-REF', 'EEG T6-REF', 
               'EEG A1-REF', 'EEG A2-REF', 'EEG FZ-REF', 'EEG CZ-REF', 
               'EEG PZ-REF', 'EEG T1-REF', 'EEG T2-REF', 'EEG EKG1-REF']
C = len(ref_headers)

signal_labels_dict = dict()

for i, edf_file in enumerate(tqdm(glob.glob(os.path.join(origin_path, '*/*/*/*/*/*/*.edf')))):
    if os.path.basename(edf_file) in ['00006563_s010_t000.edf', 
                                      '00006904_s004_t000.edf', 
                                      '00006904_s005_t000.edf']:
        continue
        
    signals, signal_headers, edf_header = pyedflib.highlevel.read_edf(edf_file)
    
    for sh in signal_headers:
        signal_labels_dict[sh['label']] = signal_labels_dict.get(sh['label'], 0) + 1

    signal_array = np.zeros((C, *signals[0].shape), dtype=np.float32)        
    
    for k, ch in enumerate(ref_headers):
        idx = -1
        for kk, sh in enumerate(signal_headers):
            if ch == sh['label']:
                idx = kk
        
        if idx < 0:
            raise ValueError(f"{edf_file} \n\t does not have {ch} label.")
        signal_array[k] = signals[idx]
    signals = signal_array
    
    train_or_eval = edf_file.split('edf')[1].split('01_tcp_ar')[0].split('\\')[1]
    pathology = edf_file.split('edf')[1].split('01_tcp_ar')[0].split('\\')[2]
    memmap_file = os.path.join(desired_path, train_or_eval, pathology, os.path.basename(edf_file).split('.edf')[0] + '.dat')
    os.makedirs(os.path.dirname(memmap_file), exist_ok=True)
    fp = np.memmap(memmap_file, 
                   dtype='float32', mode='w+', shape=signals.shape)
    fp[:] = signals[:]
    fp.flush()

print(signal_labels_dict)

In [ ]:
np.memmap(r"C:\Users\Minjae\Desktop\EEG_Project\local\dataset\tuab\eval\abnormal\00000768_s003_t000_00.dat", 
          dtype="float32", mode="r")  == np.memmap(r"C:\Users\Minjae\Desktop\EEG_Project\local\dataset\tuab\eval\abnormal\00000768_s003_t000.dat", 
          dtype="float32", mode="r")

In [ ]:
import os
import glob
import pyedflib
import numpy as np
from tqdm import tqdm

ref_headers = ['EEG FP1-REF', 'EEG FP2-REF', 'EEG F3-REF', 'EEG F4-REF',
               'EEG C3-REF', 'EEG C4-REF', 'EEG P3-REF', 'EEG P4-REF',
               'EEG O1-REF', 'EEG O2-REF', 'EEG F7-REF', 'EEG F8-REF',
               'EEG T3-REF', 'EEG T4-REF', 'EEG T5-REF', 'EEG T6-REF',
               'EEG A1-REF', 'EEG A2-REF', 'EEG FZ-REF', 'EEG CZ-REF',
               'EEG PZ-REF', 'EEG T1-REF', 'EEG T2-REF', 'EEG EKG1-REF']

signal_labels_dict = {}

for i, edf_file in enumerate(tqdm(glob.glob(os.path.join(origin_path, '*/*/*/*/*/*/*.edf')))):
    if os.path.basename(edf_file) in ['00006563_s010_t000.edf',
                                      '00006904_s004_t000.edf',
                                      '00006904_s005_t000.edf']:
        continue

    signals, signal_headers, edf_header = pyedflib.highlevel.read_edf(edf_file)

    for sh in signal_headers:
        signal_labels_dict[sh['label']] = signal_labels_dict.get(sh['label'], 0) + 1

    signal_array = np.zeros((len(ref_headers), *signals[0].shape), dtype=np.float32)
    label_indices = {label: i for i, label in enumerate(ref_headers)}

    for kk, sh in enumerate(signal_headers):
        idx = label_indices.get(sh['label'], -1)
        if idx >= 0:
            signal_array[idx] = signals[kk]

    train_or_eval, pathology, _ = edf_file.split('edf')[1].split('01_tcp_ar')[0].split('\\')[1:4]
    memmap_file = os.path.join(desired_path, train_or_eval, pathology, os.path.basename(edf_file).split('.edf')[0] + '.dat')
    os.makedirs(os.path.dirname(memmap_file), exist_ok=True)
    signal_array.tofile(memmap_file)

print(signal_labels_dict)


---
## Dataset and loader

In [ ]:
config = {}
config['dataset_path'] = './local/dataset/tuab/'
config['file_format'] = 'memmap'
config['seq_length'] = 2500
config['EKG'] = 'O'
config['device'] = 'cpu'
config['minibatch'] = 256
config['input_norm'] = 'dataset'
config['use_age'] = 'no'
train_loader, val_loader, test_loader, multicrop_test_loader = build_dataset_for_tuab_train(config, verbose=True)

In [ ]:
print(len(train_loader.dataset), len(val_loader.dataset), len(test_loader.dataset))

In [ ]:
train_loader.dataset[0]